In [5]:
def importdata():
    df = pd.read_csv('../data/df_points_2018_2020_h8.csv')
    df = df[['bike_id','lat','lng','timestood','time_start','time_end','hex_id']]
    
    df['time_start'] = pd.to_datetime(df['time_start'])
    
    return df

In [6]:
def create_tsdata(_year,_days):
    rng = pd.date_range(f'{_year}-01-01', periods=_days, freq='D')
    tsdata = pd.DataFrame({ 'date': rng, 'idle_time' : np.nan, 'list': np.nan, 'bike_count': np.nan}) 
    return tsdata.set_index('date')

In [7]:
def plot_details(tsdata,data_7d_rol,data_2m_rol):
    fig, ax = plt.subplots(figsize = (20,10))
    
    # plotting daily data
    ax.plot(tsdata['idle_time'], marker='.', markersize=2, color='0.6',linestyle='None', label='Daily')
    # plotting 7-day rolling data
    ax.plot(data_7d_rol['idle_time'], linewidth=2, label='7-d Rolling Mean')
    # plotting 2-Month rolling data -trend-
    ax.plot(data_2m_rol['idle_time'], color='0.2', linewidth=3, label='Trend (2M)')
    ax.legend()
    ax.set_xlabel('day')
    ax.set_ylabel('timestood (min)')
    ax.set_title(f'Trends in idel time of bikes in hex: --- all leipzig ---')
    
    ax2 = ax.twinx()
    ax2.plot(data_7d_rol['bike_count'],linewidth=1, color='0.5',label='Bike Count 7-d RM')
    ax2.set_xlabel('day')
    ax2.set_ylabel('bikes in hex per day')
    ax2.legend()
    
    plt.show()

In [8]:
def all_add_rows_to_days(tsdata,df):
    # iteriere über alle Tage
    for date in tqdm(tsdata.index):
        list = []
        # Adde alle Rows in die List wenn Tag = Tag
        for ind in df.index:
            if date.date() == df['time_start'][ind].date():
                list.append(df['timestood'][ind])
        # Setze Mittelwert für Tag
        if len(list) == 0:
            tsdata['bike_count'][date] = 0
            continue
        else:
            tsdata['bike_count'][date] = len(list)
            tsdata['idle_time'][date] = statistics.mean(list)
    return tsdata

# Main

In [ ]:
df = importdata()
alltsdata = create_tsdata()
alltsdata = all_add_rows_to_days(alltsdata,df)

columns_for_tsdata = ['idle_time','bike_count']
all_data_7d_rol = alltsdata[columns_for_tsdata].rolling(window = 7, center = True).mean()
all_data_2m_rol = alltsdata[columns_for_tsdata].rolling(window = 60, center = True).mean()

plot_details(alltsdata,all_data_7d_rol,all_data_2m_rol)